# Abstract

Make the test comparison associations

# Environment

In [ ]:
# Setup debugging log
import logging
logger = logging.getLogger('jwst.associations')
handler = logging.StreamHandler()
logger.addHandler(handler)
handler.setLevel(logging.DEBUG)
logger.setLevel(logging.DEBUG)

In [ ]:
from functools import partial
from glob import glob
from multiprocessing import Pool
from os import (
    listdir, 
    mkdir,
    path
)
import pdb
import re
import shutil
from tempfile import TemporaryDirectory

In [ ]:
import numpy as np

In [ ]:
from jwst.associations import (
    AssociationRegistry,
    generate
)
from jwst.associations.main import Main

In [ ]:
from jwst.associations.tests.helpers import (
    combine_pools,
    registry_level3_only,
    registry_level2_only,
    t_path,
)

In [ ]:
from jwst.associations.tests.test_standards import (
    DEF_ARGS,
    LV2_ONLY_ARGS,
    LV3_ONLY_ARGS,
    MakePars,
    standards
)

In [ ]:
# Folders
DATA_DIR = path.join(
    '..', 'data'
)
NEW_ASN_DIR = path.join(
    DATA_DIR,
    'new_asn_standards'
)
SDP_DIR = path.join(
    '..', 'data', 'sdp'
)
SDP_POOLS = path.join(
    SDP_DIR, 'pools'
)
SDP_NEW_ASN_DIR = path.join(
    SDP_DIR, 'new_asns'
)

# Library

In [ ]:
def make_standards(pool_root, 
                   main_args=None,
                   source=DATA_DIR,
                   outdir=NEW_ASN_DIR
                  ):
    """Make the association standards for a pool"""
    pool_path = path.join(
        source,
        pool_root + '.csv'
    )
    pool = combine_pools(pool_path)
    if main_args is None:
        main_args = []
    with TemporaryDirectory() as tmp_path:
        args = main_args + ['-v', '-p', tmp_path]
        results = Main(
            args,
            pool=pool
        )
        asn_paths = listdir(tmp_path)
        for asn_path in asn_paths:
            matches = re.match('[^-]+(-.+)', asn_path)
            asn_tail = matches.group(1)
            from_path = path.join(tmp_path, asn_path)
            to_path = path.join(outdir, pool_root + asn_tail)
            shutil.move(from_path, to_path)
        pool.write(path.join(outdir, pool_root + '_std.csv'))

# Main

## Clear result folder

In [ ]:
try:
    shutil.rmtree(NEW_ASN_DIR)
except FileNotFoundError:
    pass

mkdir(NEW_ASN_DIR)

In [ ]:
try:
    shutil.rmtree(SDP_NEW_ASN_DIR)
except FileNotFoundError:
    pass

mkdir(SDP_NEW_ASN_DIR)

## Make the standards

In [ ]:
make_standards_args = [
    (standard.pool_root, standard.main_args)
    for standard in standards
]

In [ ]:
def standards_map_func(t):
    make_standards(t[0], t[1], source=DATA_DIR, outdir=NEW_ASN_DIR)

In [ ]:
with Pool(6) as mp:
    mp.map(standards_map_func, make_standards_args)

### Specific cases

In [ ]:
#make_standards_args[11]

In [ ]:
standards_map_func(make_standards_args[11])

In [ ]:
#pool_006 = combine_pools('../data/pool_006_spec_nirspec.csv')

In [ ]:
#pool_006.show_in_browser(jsviewer=True)

## Make the SPD-based associations

In [ ]:
pools = glob(path.join(SDP_POOLS, '*.csv'))

In [ ]:
make_spd_args = [
    (path.splitext(path.split(pool)[1])[0], path.split(pool)[0])
    for pool in pools
]

In [ ]:
def spd_map_func(t):
    make_standards(t[0], source=t[1], outdir=SDP_NEW_ASN_DIR)

In [ ]:
with Pool(6) as mp:
    mp.map(spd_map_func, make_spd_args)

### Specific cases

In [ ]:
# make_standard_args

In [ ]:
# make_standard_args[-2]

In [ ]:
# map_func(make_standard_args[-2])

In [ ]:
# %debug